**Step1** - 
**Tinder** **API**

1. See the tinder demo
2. List all the functions which we want to build
3. Setup DB
4. Start building Flask API
5. Host the API on Heroku


**Step2** -
**Basic prerequisites to take into consideration while creating Tinder API :**
1. Login via mobile number using OTP -->
2. Create profile with -->
   1. Name
   2. Number
   3. Location
      1. Lattitude & Longitude
      2. City
      3. State
      4. Country
   4. Gender
   5. Job
   6. Passion
   7. Company
   8. Age/DOB
   9. Images * 9
   -------------
   10. UID (User Id)
   11. CreatedAt
   12. LastOpenedAt 
3. Feed -->
   1. User Data Array []
4. Matching -->
   1. Left Swipe --> Rejecting
   2. Right Swipe --> I want to talk!



**Step3**
1. https://console.firebase.google.com/
2. Auth
3. Database --> Firestore
4. Images Storing --> Firebase Storage

Firebase Authentication is used for authenticating the user using multiple Auth providers, and default Email Id/Password


**Step4**
Make the Auth system for Login and SignUp using API

**Step5** -
Features we want to build
1. Login
2. Sign 
3. Update User-Profile
4. Feed
5. Matching

In [58]:
#Auth -->
import firebase_admin

In [59]:
# Import requirements
from firebase_admin import auth, firestore, storage, credentials

In [3]:
# Credentials
cred = credentials.Certificate("/content/tinder-prathima-firebase-adminsdk-d98jh-32c7e606eb.json")
firebase_app = firebase_admin.initialize_app(cred)
store = firestore.client()

In [60]:
# Auth's Code
user = auth.create_user(
    email='prathimanew@gmail.com',
    email_verified=False,
    password='234567')
print('Sucessfully created new user: {0}'.format(user.uid))

Sucessfully created new user: vyXCt76i19X0UiIP0MWgg0nQKCT2


In [61]:
# User Login
user = auth.get_user_by_email('prathimanew@gmail.com')
print(user.uid)

vyXCt76i19X0UiIP0MWgg0nQKCT2


In [62]:
# SignUp
def signUp(emailOfUser, passwordOfUser):
  uid = ""
  message = ""
  try:
    user = auth.create_user(
        email = emailOfUser,
        email_verified = False,
        password = passwordOfUser)
    message = "Successsfully Created New User"
    uid = user.uid
  except:   
    message = "User Already There"

  return {"uid": uid, "message" : message}

In [63]:
signUp("newprathima@gmail.com", "345678")

{'message': 'Successsfully Created New User',
 'uid': '9wiW5zdpY7M2BnUrseQt8hzxcUf1'}

In [64]:
# Login
# User can login with just entering email id, this is due to Firebase Auth is implemented natively.
def login(emailOfUser, passwordOfUser):
  uid = ""
  message = ""
  try:
    user = auth.get_user_by_email(emailOfUser)
    message = "Successsfully Created New User"
    uid = user.uid
  except:   
    message = "User Not There in Firebase!"

  return {"uid": uid, "message" : message}

In [65]:
login("prathimanew@gmail.com", "234567")

{'message': 'Successsfully Created New User',
 'uid': 'vyXCt76i19X0UiIP0MWgg0nQKCT2'}

In [66]:
# Update User Data
def updateUserData(uid,dit):

  dit_user_details= {}
  dit_user_details["name"] = dit["name"]
  dit_user_details["email"] = dit["email"]
  dit_user_details["number"] = dit["number"]
  dit_user_details["image"] = dit["image"]
  dit_user_details["desp"] = dit["desp"]
  dit_user_details["dob"] = dit["dob"]
  dit_user_details["gender"] = dit["gender"]
  dit_user_details["passion"] = dit["passion"]
  dit_user_details["job"] = dit["job"]
  dit_user_details["company"] = dit["company"]
  dit_user_details["location"] = dit["location"]
  dit_user_details["createdAt"] = firestore.SERVER_TIMESTAMP

  store.collection("users").document(uid).set(dit_user_details) #it stores the data into firebase

In [73]:
# Add User Details
dit = {}
	
dit['name'] = "Prath"
dit['email'] = "xyz@gmail.com"
dit['number'] = "2345672591"
dit['image'] = "imageURL"
dit['desp'] = "single"
dit['location'] = {"coordinate":{"lat":17.431749, "lng":78.448250},
						"city":"Bangalore",
						"state":"Karnataka",
						"country":"India"}
dit['dob'] = "20/12/1989"
dit['gender'] = "female"
dit['passion'] = "travel"
dit['job'] = "none"
dit['company'] = "none"

In [74]:
updateUserData("vyXCt76i19X0UiIP0MWgg0nQKCT2", dit)

In [75]:
# Get the Feed
def getFeed(country):
  docs = store.collection("users").stream()

  dit ={}
  for doc in docs:
    if doc.to_dict().get("location").get("country") == country:
      dit[doc.id] = doc.to_dict()

  return dit

In [76]:
allProfiles = getFeed("India")

In [77]:
allProfiles

{'shZjwL5hLYVxLmycX7YnULT3BIg1': {'company': 'none',
  'createdAt': DatetimeWithNanoseconds(2021, 3, 22, 12, 49, 30, 512000, tzinfo=<UTC>),
  'desp': 'single',
  'dob': '06/04/1986',
  'email': 'xyz@gmail.com',
  'gender': 'female',
  'image': 'imageURL',
  'job': 'none',
  'location': {'city': 'Hyderabad',
   'coordinate': {'lat': 17.431749, 'lng': 78.44825},
   'country': 'India',
   'state': 'Telangana'},
  'name': 'Prathima',
  'number': '2345672951',
  'passion': 'coding'},
 'vyXCt76i19X0UiIP0MWgg0nQKCT2': {'company': 'none',
  'createdAt': DatetimeWithNanoseconds(2021, 3, 22, 13, 49, 39, 778000, tzinfo=<UTC>),
  'desp': 'single',
  'dob': '20/12/1989',
  'email': 'xyz@gmail.com',
  'gender': 'female',
  'image': 'imageURL',
  'job': 'none',
  'location': {'city': 'Bangalore',
   'coordinate': {'lat': 17.431749, 'lng': 78.44825},
   'country': 'India',
   'state': 'Karnataka'},
  'name': 'Prath',
  'number': '2345672591',
  'passion': 'travel'}}

In [79]:
# Swipe
# When profile appears, right or left swipe it. Whatever we do, data should be stored.
def swipeFun(uidA, uidB, isA_Yes, isB_Yes, firstTime):
	dit = {}

	dit['UID_A'] = uidA
	dit['UID_B'] = uidB
	dit["isUserA_Yes"] = isA_Yes
	dit['isUserB_Yes'] = isB_Yes
	dit['SawOnce'] = firstTime
	dit['createdAt'] = firestore.SERVER_TIMESTAMP

	store.collection("swipes").add(dit)

In [80]:
uidA = "vyXCt76i19X0UiIP0MWgg0nQKCT2"
uidB = "9wiW5zdpY7M2BnUrseQt8hzxcUf1"
isA_Yes = True
isB_Yes = False

firstTime = False
swipeFun(uidA, uidB, isA_Yes, isB_Yes, firstTime)

In [81]:
# Match
def getMatchFun(uid):
	docs = store.collection("swipes").stream()

	ditSwipes = {}
	for doc in docs:
		if (doc.to_dict().get("UID_A") == uid or doc.to_dict().get("UID_B") == uid) and (doc.to+dict().get("isUserA_Yes") == True and doc.to_dict().get().get("isUserB_Yes") == True):
			ditSwipes[doc.id] = doc.to_dict()


	return ditSwipes

In [ ]:
getMatchFun("9wiW5zdpY7M2BnUrseQt8hzxcUf1")